In [1]:
import parmed as pmd
import numpy as np
import os
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import colorConverter
import seaborn as sns

%load_ext autoreload
%autoreload 2
%matplotlib inline

/home/dslochower/data/applications/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/dslochower/data/applications/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/dslochower/data/applications/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Parameter comparisons

I find there are a different number of dihedral parameters applied to a system parameterized with GAFF v1.7 and SMIRNOFF99Frosst.

## Method 1 (use ParmEd)

In [6]:
smirnoff_prmtop = pmd.load_file("systems/a-bam-p/smirnoff/a000/hg.prmtop")
smirnoff_prmtop = smirnoff_prmtop[":MGO"]

gaff_prmtop = pmd.load_file("systems/a-bam-p/bgbg-tip3p/hg.topo")
gaff_prmtop = gaff_prmtop[":MGO"]

In [11]:
print(f"SMIRNOFF99Frosst: {len(smirnoff_prmtop.dihedrals)} dihedrals and {len(smirnoff_prmtop.impropers)} impropers.")

SMIRNOFF99Frosst: 630 dihedrals and 0 impropers.


In [12]:
print(f"GAFF v1.7: {len(gaff_prmtop.dihedrals)} dihedrals and {len(gaff_prmtop.impropers)} impropers.")

GAFF v1.7: 624 dihedrals and 0 impropers.


In [33]:
def find_dihedrals(structure):
    df = pd.DataFrame()
    for dihedral in structure.dihedrals:
        df = df.append(
            pd.DataFrame(
                {
                    "atom1": dihedral.atom1.name,
                    "atom2": dihedral.atom2.name,
                    "atom3": dihedral.atom3.name,
                    "atom4": dihedral.atom4.name,
                    "phi_k": dihedral.type.phi_k,
                    "per": dihedral.type.per,
                    "phase": dihedral.type.phase,
                },
                index=[0],
            ),
            ignore_index=True,
        )
    return df

In [37]:
gaff_prmtop_dihedrals = find_dihedrals(gaff_prmtop)
assert len(gaff_prmtop.dihedrals) == len(gaff_prmtop_dihedrals)
gaff_prmtop_dihedrals.drop_duplicates(inplace=True)

In [40]:
print(f"GAFF v1.7: {len(gaff_prmtop_dihedrals)} unique dihedrals.")

GAFF v1.7: 127 unique dihedrals.


In [41]:
smirnoff_prmtop_dihedrals = find_dihedrals(smirnoff_prmtop)
assert len(smirnoff_prmtop.dihedrals) == len(smirnoff_prmtop_dihedrals)
smirnoff_prmtop_dihedrals.drop_duplicates(inplace=True)

In [43]:
print(f"SMIRNOFF99Frosst: {len(smirnoff_prmtop_dihedrals)} unique dihedrals.")

SMIRNOFF99Frosst: 131 unique dihedrals.


Where are they different?

In [55]:
dihedral_differences = smirnoff_prmtop_dihedrals.merge(gaff_prmtop_dihedrals, 
                                                       indicator=True,
                                                       suffixes=("_smirnoff", "_bgbg_tip3p"),
                                                       how="outer")

In [63]:
all_differences = dihedral_differences[dihedral_differences["_merge"] != "both"]
all_differences.head()

,atom1,atom2,atom3,atom4,phi_k,per,phase,_merge
2,O1,C1,C2,C3,0.156,3,0.0,left_only
4,O5,C1,C2,O2,1.175,2,0.0,left_only
5,O5,C1,C2,C3,0.156,3,0.0,left_only
8,O2,C2,C3,C4,0.156,3,0.0,left_only
9,C1,C2,C3,O3,0.156,3,0.0,left_only


In [70]:
smirnoff_only = all_differences[all_differences["_merge"] == "left_only"]
gaff_only = all_differences[all_differences["_merge"] == "right_only"]

In [71]:
print(f"SMIRNOFF99Frosst only: {len(smirnoff_only)} dihedrals.")
print(f"GAFF v1.7 only: {len(gaff_only)} dihedrals.")

SMIRNOFF99Frosst only: 52 dihedrals.
GAFF v1.7 only: 48 dihedrals.


Because of periodicity, some sets of four atoms are listed multiple times. **I am interested in where there are any set of four atoms for which one force field applies a dihedral and the other force field does not.** If we ignore periodicity are there a different number of SMIRNOFF99Frosst-only and GAFF v1.7-only dihedrals?

In [78]:
smirnoff_tmp = smirnoff_only.drop(columns=["_merge"])
gaff_tmp = gaff_only.drop(columns=["_merge"])

dihedral_atom_name_merge = smirnoff_tmp.merge(gaff_tmp,
                           on=["atom1", "atom2", "atom3", "atom4"],
                           indicator=True,
                           suffixes=("_smirnoff", "_bgbg_tip3p"),
                           how="outer")

In [82]:
dihedral_atom_name_merge[dihedral_atom_name_merge["_merge"] != "both"]

,atom1,atom2,atom3,atom4,phi_k_smirnoff,per_smirnoff,phase_smirnoff,phi_k_bgbg_tip3p,per_bgbg_tip3p,phase_bgbg_tip3p,_merge
1,O5,C1,C2,O2,1.175,2,0.0,NaN,NaN,NaN,left_only
2,O5,C1,C2,C3,0.156,3,0.0,NaN,NaN,NaN,left_only
46,O5,C1,C2,H2,0.250,1,0.0,NaN,NaN,NaN,left_only


It seems that only SMIRNOFF99Frosst applies dihedrals to these four atoms.

In [87]:
name_type_map = dict()
for atom in gaff_prmtop.atoms:
    name_type_map[atom.name] = atom.type

In [90]:
for index, row in dihedral_atom_name_merge[dihedral_atom_name_merge["_merge"] != "both"].iterrows():
    print(f'{name_type_map[row["atom1"]]}-{name_type_map[row["atom2"]]}-{name_type_map[row["atom3"]]}-{name_type_map[row["atom4"]]}')

os-c3-c3-oh
os-c3-c3-c3
os-c3-c3-h1


I can confirm that I don't see these dihedrals in the `frcmod` file.

** I think there may also be SCEE/SCNB scaling differences.**